In [8]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("yarn").getOrCreate()

In [9]:
from pyspark.sql.functions import explode, collect_list, size, col, row_number, sort_array,udf, count, desc
from pyspark.sql import Window, types
from itertools import combinations

In [10]:
graphPath = "/data/graphDFSample"

In [11]:
def emit(users):
    return combinations(sorted(users), 2)

pairschema = types.StructType([
    types.StructField("u1", types.IntegerType(), False),
    types.StructField("u2", types.IntegerType(), False)
])

udf_emit = udf(emit, types.ArrayType(pairschema))

reversedGraph = (sparkSession.read.parquet(graphPath)
                 .withColumn("friend", explode('friends'))
                 .groupBy("friend")
                 .agg(collect_list("user").alias("users"))
                 .withColumn("users", sort_array('users'))
                 .where(size('users') > '1')
                )

In [12]:
pairs = (reversedGraph.withColumn("pairs", udf_emit('users'))
         .withColumn("pair", explode('pairs'))
         .groupBy("pair")
         .agg(count("pair").alias("pair_count"))
        )

In [13]:
results = (pairs.select(col("pair_count"), "pair.*")
           .orderBy(desc("pair_count"), desc("u1"), desc("u2"))
           .limit(49)
          ).collect()

In [14]:
for pair, u1, u2 in results:
    print("{} {} {}".format(pair, u1, u2))

3206 27967558 42973992
3130 20158643 42973992
3066 22582764 42973992
3044 21864412 51640390
3021 17139850 51640390
3010 14985079 51640390
2970 17139850 21864412
2913 20158643 27967558
2903 22280814 51151280
2870 23848749 51640390
2855 20158643 22582764
2849 20158643 44996025
2846 22280814 42973992
2784 21864412 23848749
2779 31964081 51640390
2776 39205988 51640390
2754 17139850 23848749
2749 22582764 27967558
2728 50561859 51640390
2724 15485897 51640390
2700 28135661 42973992
2655 22280814 27967558
2653 42973992 43548989
2639 26755857 51640390
2621 14635589 51640390
2608 15485897 17139850
2606 17139850 26755857
2601 21864412 39205988
2600 8406745 51640390
2599 37735419 51640390
2597 20158643 28135661
2585 40003405 42973992
2585 21864412 31964081
2581 27967558 43548989
2579 23848749 31964081
2578 27967558 28135661
2578 15485897 21864412
2577 42973992 64755069
2574 51151280 57077210
2573 20158643 43548989
2566 21864412 26755857
2564 22280814 64755069
2561 42973992 44996025
2556 1713985